In [1]:
import sys
sys.path.append('..')
from core import computation_graph
from core import graph_merge
from visualize_graph import network_graph
from image_utilities import load_images

import run_image_experiment

%load_ext autoreload
%autoreload 2

/home/jakeval/.local/pipx/venvs/notebook/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## This single graph has redundant computation

In [309]:
@computation_graph.optex_process('return')
def transform(x):
    return x

@computation_graph.optex_process('return')
def combine(x, y, z):
    return x + y + z

@computation_graph.optex_composition('return')
def pipeline1(x):
    x.name = "input"
    t1 = transform(x)
    t1.name = "transform_1"
    t2 = transform(x)
    t2.name = "transform_2"
    t3 = transform(x)
    t3.name = "transform_3"
    c = combine(t1, t2, t3)
    c.name = 'output'
    return c

The same `transform` function is called multiple times with the same `input` argument.

In [310]:
graph = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(pipeline1, 'graph1'))
network_graph.Network_Graph(graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## We can merge the graph with itself

This eliminates the redundant computation

In [311]:
inputs = {
    computation_graph.Artifact(10): {
        graph.name: [graph.inputs[0]]
    }
}

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


Note that the edges from `transform_3` to `combine` are maintained in the graph structure, but aren't represented in the graph visualization.

In [288]:
print(merged_inputs['all_inputs'][0].name)
print("==>")
print(merged_inputs['all_inputs'][0].children[0].name)
print("==>")
print(merged_inputs['all_inputs'][0].children[0].children[0].name)
print("==>")

merged_transform = merged_inputs['all_inputs'][0].children[0].children[0]

print([f'{role}={child.name}' for role, child in zip(merged_transform.children_roles, merged_transform.children)])

artifact
==>
transform
==>
transform_3
==>
['z=combine', 'y=combine', 'x=combine']


## We can also merge multiple graphs

Although graph1 and graph2 are identical, they were defined as different graphs

In [289]:
@computation_graph.optex_process('output')
def dual_arg(arg1, arg2):
    return arg1*10 + arg2


graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "graph1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "graph2"))

unmerged_graph = computation_graph.EdgeGraph.from_output_artifacts(graph1.outputs + graph2.outputs, name='unmerged_graph')
network_graph.Network_Graph(unmerged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [312]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "graph1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "graph2"))

arg1 = computation_graph.Artifact(10)
arg2 = computation_graph.Artifact(20)
inputs = graph_merge.get_inputs([
    (graph1, dual_arg, 'arg1', arg1),
    (graph1, dual_arg, 'arg2', arg2),
    (graph2, dual_arg, 'arg1', arg1),
    (graph2, dual_arg, 'arg2', arg2)])


merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


If the arguments are mismatched so their roles don't align, the graphs can't be merged.

In [313]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "g2"))

arg1 = computation_graph.Artifact(10)
arg1.name = 'arg1'
arg2 = computation_graph.Artifact(20)
arg2.name = 'arg2'
inputs = graph_merge.get_inputs([
    (graph1, dual_arg, 'arg2', arg1),   # Call graph1 with arg1=arg2 and arg2=arg1
    (graph1, dual_arg, 'arg1', arg2),   
    (graph2, dual_arg, 'arg1', arg1),   # Call graph2 with arg1=arg1 and arg2=arg2
    (graph2, dual_arg, 'arg2', arg2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


Merging is also impossible even if only one of the arguments doesn't match.

In [314]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_arg, "g2"))

arg1_merged = computation_graph.Artifact(10)
arg2_graph1 = computation_graph.Artifact(20)
arg2_graph2 = computation_graph.Artifact(30)
inputs = graph_merge.get_inputs([
    (graph1, dual_arg, 'arg1', arg1_merged),
    (graph1, dual_arg, 'arg2', arg2_graph1),
    (graph2, dual_arg, 'arg1', arg1_merged),
    (graph2, dual_arg, 'arg2', arg2_graph2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### Let's try a more complex graph

In [321]:
@computation_graph.optex_process(('transform_out_1', 'transform_out_2'))
def dual_transform(arg1):
    return arg1*2, arg1+5

@computation_graph.optex_process('transform_out')
def transform(arg):
    return arg + 3

@computation_graph.optex_process('final_output')
def dual_input(arg1, arg2):
    return arg1*10 + arg2


@computation_graph.optex_composition(('output_1', 'output_2'))
def dual_return(arg1, arg2):
    arg1.name = 'arg1'
    arg2.name = 'arg2'
    out_1, out_2 = dual_transform(arg1)
    out_1.name = 'transform_out_1'
    out_2.name = 'transform_out_2'
    out = transform(arg2)
    out.name = 'transform_out'
    final = dual_input(out_2, out)
    final.name = 'final_out'
    return out_1, final


graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g2"))

unmerged_graph = computation_graph.EdgeGraph.from_output_artifacts(graph1.outputs + graph2.outputs, name='unmerged_graph')
network_graph.Network_Graph(unmerged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### Let's try merging when all arguments match

In [320]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g2"))

arg1 = computation_graph.Artifact(10)
arg1.name = 'arg1'
arg2 = computation_graph.Artifact(20)
arg2.name = 'arg2'
inputs = graph_merge.get_inputs([
    (graph1, dual_transform, 'arg1', arg1),
    (graph1, transform, 'arg', arg2),
    (graph2, dual_transform, 'arg1', arg1),
    (graph2, transform, 'arg', arg2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### If only some of the arguments match, only part of the graph is merged.

In [301]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g2"))


arg1_merged = computation_graph.Artifact(10)
arg1_merged.name = 'arg1_merged'
arg2_graph1 = computation_graph.Artifact(20)
arg2_graph1.name = 'arg2_graph1'
arg2_graph2 = computation_graph.Artifact(30)
arg2_graph2.name = 'arg2_graph2'

inputs = graph_merge.get_inputs([
    (graph1, dual_transform, 'arg1', arg1_merged),
    (graph1, transform, 'arg', arg2_graph1),
    (graph2, dual_transform, 'arg1', arg1_merged),
    (graph2, transform, 'arg', arg2_graph2)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### We can merge many graphs at a time

In [324]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g2"))
graph3 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g3"))


arg1_merged = computation_graph.Artifact(10)
arg1_merged.name = 'arg1_merged'
arg2_graph13 = computation_graph.Artifact(20)
arg2_graph13.name = 'arg2_graph_1_3'
arg2_graph2 = computation_graph.Artifact(30)
arg2_graph2.name = 'arg2_graph_2'

inputs = graph_merge.get_inputs([
    (graph1, dual_transform, 'arg1', arg1_merged),
    (graph1, transform, 'arg', arg2_graph13),
    (graph2, dual_transform, 'arg1', arg1_merged),
    (graph2, transform, 'arg', arg2_graph2),
    (graph3, dual_transform, 'arg1', arg1_merged),
    (graph3, transform, 'arg', arg2_graph13)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2, graph3], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).pyvis_graph.show('test.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### We can also execute a graph

In [306]:
graph1 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g1"))
graph2 = graph_merge.make_expanded_graph_copy(computation_graph.generate_static_graph(dual_return, "g2"))


arg1_merged = computation_graph.Artifact(10)
arg1_merged.name = 'arg1_merged'
arg2_graph1 = computation_graph.Artifact(20)
arg2_graph1.name = 'arg2_graph1'
arg2_graph2 = computation_graph.Artifact(30)
arg2_graph2.name = 'arg2_graph2'

inputs = graph_merge.get_inputs([
    (graph1, dual_transform, 'arg1', arg1_merged),
    (graph1, transform, 'arg', arg2_graph1),
    (graph2, dual_transform, 'arg1', arg1_merged),
    (graph2, transform, 'arg', arg2_graph2)])


merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2], inputs, "merged")

inputs = graph_merge.get_merged_inputs(
    merged_inputs,
    {
        10: ('g1', dual_transform, 'arg1'),
        20: ('g1', transform, 'arg'),
        30: ('g2', transform, 'arg')
    })

outputs = graph_merge.execute_graph(merged_graph, inputs)

In [307]:
for output, value in outputs.items():
    print(f"'{output.name}' in {output.agents} is {value}")

'final_out' in ['g2'] is 183
'transform_out_1' in ['g1', 'g2'] is 20
'final_out' in ['g1'] is 173


# Create and merge the experiment graphs

In [344]:
graph1 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline1.run_batch, 'g1'))

network_graph.Network_Graph(graph1, notebook=True).save_graph()

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [345]:
graph2 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline2.run_batch, 'g2'))

network_graph.Network_Graph(graph2, notebook=True).save_graph()

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [346]:
graph3 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline3.run_batch, 'g3'))

network_graph.Network_Graph(graph3, notebook=True).save_graph()

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


### Merge the graphs

In [3]:
graph1 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline1.run_batch, 'g1'))

graph2 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline2.run_batch, 'g2'))

graph3 = graph_merge.make_expanded_graph_copy(
    computation_graph.Graph.from_process(
        run_image_experiment.ImagePipeline3.run_batch, 'g3'))

batch_index_merged = computation_graph.Artifact(None)
batch_size_merged = computation_graph.Artifact(None)
spark_session_merged = computation_graph.Artifact(None)

batch_index_merged.name = 'batch_index_merged'
batch_size_merged.name = 'batch_size_merged'
spark_session_merged.name = 'spark_session_merged'

function = load_images.load_imagenet_data

inputs = graph_merge.get_inputs([
    (graph1, function, 'batch_index', batch_index_merged),
    (graph1, function, 'batch_size', batch_size_merged),
    (graph1, function, 'spark_session', spark_session_merged),
    (graph2, function, 'batch_index', batch_index_merged),
    (graph2, function, 'batch_size', batch_size_merged),
    (graph2, function, 'spark_session', spark_session_merged),
    (graph3, function, 'batch_index', batch_index_merged),
    (graph3, function, 'batch_size', batch_size_merged),
    (graph3, function, 'spark_session', spark_session_merged)])

merged_graph, merged_inputs, merged_outputs = graph_merge.merge_graphs([graph1, graph2, graph3], inputs, "merged")
network_graph.Network_Graph(merged_graph, notebook=True).save_graph()

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
